In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.data_pipeline.mongodb_manager import MongoDBManager
from iris.data_pipeline.product_handler import ProductHandler
from iris.data_pipeline.web_shop_scraper import WebShopScraper


In [ ]:
# Initialize configuration manager
config_manager = DataPipelineConfigManager() 
shop_config = config_manager.shop_config
storage_config = config_manager.storage_config
mongodb_config = config_manager.mongodb_config

# Create MongoDB manager with configuration
mongodb_manager = MongoDBManager(mongodb_config)

# Create product handler
product_handler = ProductHandler(
    shop_config=shop_config,
    storage_config=storage_config,
    mongodb_manager=mongodb_manager,
)

# Create web shop scraper
web_shop_scraper = WebShopScraper(
    shop_config=shop_config,
    product_handler=product_handler,
)

# Start scraping
with mongodb_manager as db:
    for product, images in web_shop_scraper.scrape():
        print(f"Scraped product: {product.data['title']}, Images: {len(images)}")

        db.upsert(mongodb_config.product_collection, product)
        db.upsert(mongodb_config.image_metadata_collection, images)
